In [5]:
import pandas as pd

# Load the Excel file
file_path = file_path = '/Users/kavita/Desktop/MLProject/casestudy.xlsx'  
data = pd.read_excel(file_path, sheet_name='input_1_histrocial')

# Filter for May orders based on the 'date_order' column
may_orders = data[data['date_order'].dt.month == 5]

# Group by 'date_order' and 'day_of_week_order', then sum the 'items'
May_order_items = may_orders.groupby(['date_order', 'day_of_week_order'])['items'].sum().reset_index()

# Rename columns for clarity
May_order_items.columns = ['date_order', 'day_of_week_order', 'total_items']

# Display the summary
print(May_order_items.head())


  date_order  day_of_week_order  total_items
0 2022-05-01                  7       104200
1 2022-05-02                  1       105356
2 2022-05-03                  2       102407
3 2022-05-04                  3        83761
4 2022-05-05                  4        59848


In [6]:

file_path_2 = '/Users/kavita/Desktop/MLProject/casestudy.xlsx'  

# Load the sheets into DataFrames
historical_data = pd.read_excel(file_path_2, sheet_name='input_1_histrocial')

In [7]:
jan_to_apr_data = historical_data[(historical_data['date_order'].dt.month >= 1) & 
                                  (historical_data['date_order'].dt.month <= 4)].copy()

jan_to_apr_data['lag_days'] = (jan_to_apr_data['date_wh_receive'] - jan_to_apr_data['date_order']).dt.days


In [8]:
# Filter out records where lag days exceed 15, as after that item count is 0 in most cases
filtered_data = jan_to_apr_data[jan_to_apr_data['lag_days'] <= 15]

# Group by lag days and sum up items for each lag day
lag_days_with_items_distribution = filtered_data.groupby('lag_days')['items'].sum()

# Calculate the total items received across all considered lag days
total_items = lag_days_with_items_distribution.sum()

# Calculate the percentage of total items received for each lag day
distribution_percentages = (lag_days_with_items_distribution / total_items * 100).reset_index()

# distribution_percentages['items'] = distribution_percentages['items'].round(2)

distribution_percentages.columns = ['receipt_diff', 'items']

# reading this percentage
distribution_percentages.head(15)
# print(distribution_percentages)

,receipt_diff,items
0,0,46.933192
1,1,20.224993
2,2,25.474495
3,3,5.161883
4,4,1.321446
5,5,0.466414
6,6,0.114265
7,7,0.134364
8,8,0.077625
9,9,0.040049


In [9]:
forecast_dates = pd.date_range(start=May_order_items['date_order'].min(), 
                               end=May_order_items['date_order'].max() 
                               + pd.Timedelta(days=15))

In [10]:
forecast_distribution = pd.DataFrame(index=forecast_dates, columns=['forecasted_wh_received_items'])
forecast_distribution['forecasted_wh_received_items'] = 0  # Initialize all values to 0


In [11]:

# Applying the historical distribution to each day's forecasted sales
for _, row in May_order_items.iterrows():
    for _, dist in distribution_percentages.iterrows():
        forecast_date = row['date_order'] + pd.Timedelta(days=int(dist['receipt_diff']))
        if forecast_date in forecast_distribution.index:
            forecast_distribution.at[forecast_date, 'forecasted_wh_received_items'] += row['total_items'] * (dist['items'] / 100)

            
forecast_distribution.reset_index(inplace=True)
forecast_distribution.rename(columns={'index': 'date_wh_receive'}, inplace=True)

In [12]:
display(forecast_distribution.round())

,date_wh_receive,forecasted_wh_received_items
0,2022-05-01,48904.0
1,2022-05-02,70521.0
2,2022-05-03,95916.0
3,2022-05-04,92241.0
4,2022-05-05,77932.0
5,2022-05-06,67596.0
6,2022-05-07,57963.0
7,2022-05-08,102379.0
8,2022-05-09,83556.0
9,2022-05-10,93794.0


In [13]:
# Save the forecast to a csv file
forecast_distribution.to_csv("forecast_distribution_may_2024_lagD.csv", index=False)
from IPython.display import FileLink
FileLink('forecast_distribution_may_2024_lagD.csv')

/Users/kavita/forecast_distribution_may_2024_lagD.csv